In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc
import os
print(os.getcwd())


/home/tian/Dropbox/2017/kaggle/zillow/output


In [2]:
train = pd.read_csv('../input/train_2016_v2.csv')


In [3]:
prop = pd.read_csv('../input/properties_2016.csv')

/home/tian/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

In [5]:
df_train = train.merge(prop, how='left', on='parcelid')


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90275 entries, 0 to 90274
Data columns (total 60 columns):
parcelid                        90275 non-null int64
logerror                        90275 non-null float64
transactiondate                 90275 non-null object
airconditioningtypeid           28781 non-null float32
architecturalstyletypeid        261 non-null float32
basementsqft                    43 non-null float32
bathroomcnt                     90275 non-null float32
bedroomcnt                      90275 non-null float32
buildingclasstypeid             16 non-null float32
buildingqualitytypeid           57364 non-null float32
calculatedbathnbr               89093 non-null float32
decktypeid                      658 non-null float32
finishedfloor1squarefeet        6856 non-null float32
calculatedfinishedsquarefeet    89614 non-null float32
finishedsquarefeet12            85596 non-null float32
finishedsquarefeet13            33 non-null float32
finishedsquarefeet15        

In [7]:
x_train = df_train.drop(['parcelid', 
                         'logerror', 
                         'transactiondate', 
                         'propertyzoningdesc', 
                         'propertycountylandusecode'
                         ], axis=1)

train_columns = x_train.columns

y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)


((90275, 55), (90275,))


In [8]:
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

In [9]:
#del df_train; gc.collect()
split = int(round(.84*df_train.shape[0],0))

split

75831

In [10]:

x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

In [11]:
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)


In [12]:
#stolen tuning from someone elses notebok
params = {}
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['num_leaves'] = 31
params['min_data'] = 400
params['min_data_in_leaf'] = 450
params['min_hessian'] = 1
params['feature_fraction'] = .5

In [13]:
params2 = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 70,
    'learning_rate': 0.0005,
    'min_data': 300,
    'min_hessian': 1,
    'feature_fraction':.6
}

In [ ]:
watchlist = [d_valid]


In [ ]:
clf = lgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200)

[1]	valid_0's l1: 0.0649885
Train until valid scores didn't improve in 200 rounds.
[2]	valid_0's l1: 0.0649875
[3]	valid_0's l1: 0.0649868
[4]	valid_0's l1: 0.0649861
[5]	valid_0's l1: 0.0649853
[6]	valid_0's l1: 0.0649838
[7]	valid_0's l1: 0.0649825
[8]	valid_0's l1: 0.0649818
[9]	valid_0's l1: 0.0649806
[10]	valid_0's l1: 0.0649798
[11]	valid_0's l1: 0.0649787
[12]	valid_0's l1: 0.0649777
[13]	valid_0's l1: 0.0649765
[14]	valid_0's l1: 0.0649756
[15]	valid_0's l1: 0.0649745
[16]	valid_0's l1: 0.0649736
[17]	valid_0's l1: 0.0649728
[18]	valid_0's l1: 0.0649717
[19]	valid_0's l1: 0.0649707
[20]	valid_0's l1: 0.0649698
[21]	valid_0's l1: 0.0649686
[22]	valid_0's l1: 0.0649671
[23]	valid_0's l1: 0.0649666
[24]	valid_0's l1: 0.0649657
[25]	valid_0's l1: 0.0649645
[26]	valid_0's l1: 0.0649635
[27]	valid_0's l1: 0.0649625
[28]	valid_0's l1: 0.0649616
[29]	valid_0's l1: 0.0649604
[30]	valid_0's l1: 0.0649593
[31]	valid_0's l1: 0.0649581
[32]	valid_0's l1: 0.0649572
[33]	valid_0's l1: 0.06495

[284]	valid_0's l1: 0.0647667
[285]	valid_0's l1: 0.0647659
[286]	valid_0's l1: 0.0647654
[287]	valid_0's l1: 0.064765
[288]	valid_0's l1: 0.0647645
[289]	valid_0's l1: 0.0647635
[290]	valid_0's l1: 0.0647628
[291]	valid_0's l1: 0.0647622
[292]	valid_0's l1: 0.0647616
[293]	valid_0's l1: 0.0647612
[294]	valid_0's l1: 0.0647609
[295]	valid_0's l1: 0.0647604
[296]	valid_0's l1: 0.0647599
[297]	valid_0's l1: 0.0647592
[298]	valid_0's l1: 0.0647586
[299]	valid_0's l1: 0.0647578
[300]	valid_0's l1: 0.0647572
[301]	valid_0's l1: 0.0647566
[302]	valid_0's l1: 0.0647559
[303]	valid_0's l1: 0.0647552
[304]	valid_0's l1: 0.0647548
[305]	valid_0's l1: 0.0647542
[306]	valid_0's l1: 0.0647535
[307]	valid_0's l1: 0.064753
[308]	valid_0's l1: 0.0647525
[309]	valid_0's l1: 0.0647518
[310]	valid_0's l1: 0.0647515
[311]	valid_0's l1: 0.064751
[312]	valid_0's l1: 0.0647506
[313]	valid_0's l1: 0.0647499
[314]	valid_0's l1: 0.0647495
[315]	valid_0's l1: 0.064749
[316]	valid_0's l1: 0.0647485
[317]	valid_0'

[559]	valid_0's l1: 0.0646495
[560]	valid_0's l1: 0.0646492
[561]	valid_0's l1: 0.064649
[562]	valid_0's l1: 0.0646486
[563]	valid_0's l1: 0.0646484
[564]	valid_0's l1: 0.064648
[565]	valid_0's l1: 0.0646479
[566]	valid_0's l1: 0.0646474
[567]	valid_0's l1: 0.0646469
[568]	valid_0's l1: 0.0646467
[569]	valid_0's l1: 0.0646466
[570]	valid_0's l1: 0.0646463
[571]	valid_0's l1: 0.0646462
[572]	valid_0's l1: 0.0646459
[573]	valid_0's l1: 0.0646455
[574]	valid_0's l1: 0.0646454
[575]	valid_0's l1: 0.064645
[576]	valid_0's l1: 0.0646447
[577]	valid_0's l1: 0.0646444
[578]	valid_0's l1: 0.0646442
[579]	valid_0's l1: 0.0646439
[580]	valid_0's l1: 0.0646436
[581]	valid_0's l1: 0.0646432
[582]	valid_0's l1: 0.0646428
[583]	valid_0's l1: 0.0646423
[584]	valid_0's l1: 0.064642
[585]	valid_0's l1: 0.0646413
[586]	valid_0's l1: 0.0646408
[587]	valid_0's l1: 0.0646406
[588]	valid_0's l1: 0.0646401
[589]	valid_0's l1: 0.0646399
[590]	valid_0's l1: 0.0646398
[591]	valid_0's l1: 0.0646396
[592]	valid_0'

[837]	valid_0's l1: 0.064586
[838]	valid_0's l1: 0.064586
[839]	valid_0's l1: 0.0645858
[840]	valid_0's l1: 0.0645856
[841]	valid_0's l1: 0.0645852
[842]	valid_0's l1: 0.0645848
[843]	valid_0's l1: 0.0645845
[844]	valid_0's l1: 0.0645845
[845]	valid_0's l1: 0.0645845
[846]	valid_0's l1: 0.0645845
[847]	valid_0's l1: 0.0645842
[848]	valid_0's l1: 0.0645841
[849]	valid_0's l1: 0.0645841
[850]	valid_0's l1: 0.0645841
[851]	valid_0's l1: 0.0645842
[852]	valid_0's l1: 0.0645839
[853]	valid_0's l1: 0.0645838
[854]	valid_0's l1: 0.0645835
[855]	valid_0's l1: 0.0645833
[856]	valid_0's l1: 0.0645833
[857]	valid_0's l1: 0.0645829
[858]	valid_0's l1: 0.0645827
[859]	valid_0's l1: 0.0645826
[860]	valid_0's l1: 0.0645825
[861]	valid_0's l1: 0.0645825
[862]	valid_0's l1: 0.0645824
[863]	valid_0's l1: 0.0645823
[864]	valid_0's l1: 0.064582
[865]	valid_0's l1: 0.0645819
[866]	valid_0's l1: 0.0645815
[867]	valid_0's l1: 0.0645813
[868]	valid_0's l1: 0.0645811
[869]	valid_0's l1: 0.0645811
[870]	valid_0

[1110]	valid_0's l1: 0.0645682
[1111]	valid_0's l1: 0.0645681
[1112]	valid_0's l1: 0.064568
[1113]	valid_0's l1: 0.0645679
[1114]	valid_0's l1: 0.0645679
[1115]	valid_0's l1: 0.064568
[1116]	valid_0's l1: 0.064568
[1117]	valid_0's l1: 0.0645681
[1118]	valid_0's l1: 0.0645682
[1119]	valid_0's l1: 0.0645682
[1120]	valid_0's l1: 0.0645682
[1121]	valid_0's l1: 0.0645681
[1122]	valid_0's l1: 0.064568
[1123]	valid_0's l1: 0.064568
[1124]	valid_0's l1: 0.0645679
[1125]	valid_0's l1: 0.0645679
[1126]	valid_0's l1: 0.064568
[1127]	valid_0's l1: 0.0645679
[1128]	valid_0's l1: 0.0645678
[1129]	valid_0's l1: 0.0645677
[1130]	valid_0's l1: 0.0645676
[1131]	valid_0's l1: 0.0645676
[1132]	valid_0's l1: 0.0645676
[1133]	valid_0's l1: 0.0645676
[1134]	valid_0's l1: 0.0645677
[1135]	valid_0's l1: 0.0645678
[1136]	valid_0's l1: 0.0645676
[1137]	valid_0's l1: 0.0645677
[1138]	valid_0's l1: 0.0645677
[1139]	valid_0's l1: 0.0645677
[1140]	valid_0's l1: 0.0645677
[1141]	valid_0's l1: 0.0645678
[1142]	valid_0

In [ ]:

print("Prepare for the prediction ...")
sample = pd.read_csv('../input/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')
del sample, prop; gc.collect()
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":4})
p_test = clf.predict(x_test)

p_test = 0.97*p_test + 0.03*0.011

del x_test; gc.collect()

print("Start write result ...")
sub = pd.read_csv('../input/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('lgbm_starter.csv', index=False, float_format='%.4f')